In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import calendar

In [67]:
test_date = dt.datetime(2017,8,25)

In [80]:
df = pd.read_csv("transactions.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index(df['Date']).drop('Date',axis=1)
df.head()

,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
Date,,,,,,,,
2017-09-09,Gig Car Share,GIG CAR SHARE,0.50,debit,Rental Car & Taxi,Discover,NaN,NaN
2017-09-09,Gig Car Share,GIG CAR SHARE,0.50,debit,Rental Car & Taxi,Discover,NaN,NaN
2017-09-10,Lyft,LYFT *RIDE SUN 2PM 855-280-0278 CA,15.38,debit,Rental Car & Taxi,Discover,NaN,NaN
2017-09-10,Lyft,LYFT *RIDE SUN 4PM 855-280-0278 CA,10.88,debit,Rental Car & Taxi,Discover,NaN,NaN
2017-09-09,Cafe Durant,CAFE DURANT BERKELEY CA,9.66,debit,Restaurants,Discover,NaN,NaN


In [51]:
to = dt.datetime.today()

In [139]:
categories = set(df['Category'])
print(categories)

{'Pharmacy', 'Public Transportation', 'Tuition', 'Gift', 'Advertising', 'Sporting Goods', 'Restaurants', 'Clothing', 'Transfer for Cash Spending', 'Rental Car & Taxi', 'Television', 'Groceries', 'Credit Card Payment', 'Air Travel', 'Doctor', 'Gym', 'Fast Food', 'Alcohol & Bars', 'Food & Dining', 'Shopping', 'Mortgage & Rent', 'Hair', 'Coffee Shops'}


In [72]:
def getTodayExpenditure(tab, today=dt.datetime.today()):
    '''gets amount of money spent toady'''
    today_transactions = tab.loc[today.date()]
    return sum(today_transactions['Amount'])

In [79]:
test = getTodayExpenditure(df,test_date)
print("$",round(test,2))

$ 85.64


In [137]:
def getRangeExpenditure(tab, date_start, date_end, category=None):
    '''returns the amount of money spent in date range'''
    today_transactions = tab.loc[[i for i in pd.date_range(date_start, date_end).values if i in tab.index.values]]
    today_transactions.head()
    if (category==None):
        return sum(today_transactions['Amount'])
    else:
        return sum(today_transactions[today_transactions['Category']==category]['Amount'])

In [138]:
getRangeExpenditure(df, test_date,dt.datetime.today().date(), 'Restaurants')

89.379999999999995

In [ ]:
def getFrequencies(tab, freq='daily'):
    """returns all frequencies
    possible freq = 'daily', 'weekly', 'monthly', 'yearly' """
    for c in categories:
        

In [154]:
def getFrequencyDate(time, freq='daily'):
    """returns the start and end dates based on the given frequency
    possible freq = 'daily', 'weekly', 'monthly', 'yearly' """
    if (freq=='weekly'):
        start = time - dt.timedelta(days=time.weekday())
        end = start + dt.timedelta(days=6)
    elif (freq=='monthly'):
        start = time.replace(day=1)
        start_day, end_day = calendar.monthrange(time.year,time.month)
        end = time.replace(day=end_day)
    elif (freq=='yearly'):
        start = time.replace(month=1).replace(day=1)
        end = time.replace(month=12).replace(day=31)
    else:
        start = time
        end = time
    return start, end

In [157]:
getFrequencyDate(df, test_date,'yearly')

2017-01-01 00:00:00 2017-12-31 00:00:00


4